`otto` automatically tests functions and classes.  Defining classes should do more than change the namespace.

    %reload_ext XXX.otto
    
to install automatic testing.

In [1]:
    from unittest import *
    from doctest import *
    from ast import *
    from functools import partial
    from typing import Callable
    from dataclasses import dataclass, field

    from inspect import *
    from hypothesis import given, strategies, strategies as st, assume, HealthCheck, Verbosity, settings

    INFER = True
    def infer(object):
        spec = getfullargspec(object)
        annotations = {}
        if not spec.args:
            return FunctionTestCase(object)
        else:
            annotations.update(spec.annotations) 
            if not any(map(annotations.__contains__, spec.args)):
                # ghetto typing
                main = __import__('__main__')
                for arg in spec.args:
                    thing = eval(arg)
                    if isinstance(thing, type):
                        annotations[arg] = thing
                    else: return
            if not spec.defaults:
                return FunctionTestCase(given(**{
                    str: st.from_type(callable)
                    for str, callable in annotations.items()
                })(object))
        

In [2]:
    @dataclass
    class Testing(object):
        shell: 'ip' = field(default_factory=get_ipython)
        tests = list()
            
        def post_run_cell(Testing):
            from types import ModuleType
            global INFER
            module, main = ModuleType('__main__'), __import__('__main__')
            tests = list()
            module.__test__ = getattr(main, '__test__', {})
            if not Testing.tests: return
            while Testing.tests:
                object = Testing.tests.pop(0)
                current = str(len(module.__test__))
                if isinstance(object, Str) and '>>> ' in object.s:
                    module.__test__[current] = object.s
                if isinstance(object, ClassDef):    
                    object = getattr(main, object.name)
                    if issubclass(object, TestCase):
                        tests.append(object())
                        continue
                    if hasattr(object, 'runTest'):
                        tests.append(FunctionTestCase(partial(object.runTest, object)))
                        
                    module.__test__[current] = object
                if isinstance(object, FunctionDef):
                    object = getattr(main, object.name)
                    try:
                        if INFER:
                            _test = infer(object)
                            _test and tests.append(_test)
                    except: ...
                    if getattr(object, '__doc__', ''):
                        module.__test__[object.__name__] = object
            if tests or module.__test__:
                tests.append(DocTestSuite(module, vars(main)))
                suite = TestSuite(tests)
                TextTestRunner().run(suite)

In [3]:
    class DiscoverTests(NodeTransformer):
        def visit_star(self, node):
            Testing.tests += node,
            return node
        visit_Str = visit_ClassDef =  visit_FunctionDef = visit_star

In [4]:
    settings.register_profile('ip', settings(
        suppress_health_check=(HealthCheck.return_value,),
        verbosity=Verbosity.normal,))


In [5]:
    def load_ipython_extension(ip=get_ipython()):
        global testing
        testing = Testing(shell=get_ipython())
        ip.ast_transformers = [_ for _ in ip.ast_transformers if not isinstance(_, DiscoverTests)]+[DiscoverTests()]
        ip.events.register('post_run_cell', testing.post_run_cell)
        settings.load_profile('ip')

In [6]:
    def unload_ipython_extension(ip=get_ipython()):
        global testing
        ip.ast_transformers = [_ for _ in ip.ast_transformers if not isinstance(_, DiscoverTests)]
        ip.events.unregister('post_run_cell', testing.post_run_cell)

In [7]:
    if __name__ == '__main__':
        load_ipython_extension()
    

    !jupyter nbconvert --inplace --execute Untitled2.ipynb